# Основные определения

***Определение.*** Префикс-функция.<br>
$\Pi(s)$ $-$ максимальный суффикс строки $s$, не равный самой строке, являющийся одновременно ее префиксом.

Примеры:  
$\Pi(\text{'aba'}) = \text{'a'}$  
$\Pi(\text{'abab'}) = \text{'ab'}$  
$\Pi(\text{'ababa'}) = \text{'aba'}$  







# Задача точного поиска образца в строке

***Задача.*** Дан паттерн $P$ и текст $T$. Необходимо найти все полные вхождения паттерна $P$ в текст $T$.

In [1]:
def simple_substring_search(pattern, text):
    result = []
    for i in range(len(text) - len(pattern) + 1):
        for j in range(len(pattern)):
            if pattern[j] != text[i + j]:
                break
        else:
            result.append(i)
    return result

simple_substring_search('aba', 'ababa')

[0, 2]

# Наивный алгоритм и его сложность

Начиная с каждого символа текста $T$ производить сравнение символов теста и паттерна. Если все символы равны $-$ паттерн найден.

Сложность данного алгоритма $O(|P||T|)$

# Алгоритм КМП

Произведем последовательное вычисление длин префикс-функции для всех строк меньшей длины.

|$S$    |a|b|a|b|a|
|-------|-|-|-|-|-|
|$|\Pi|$|0|0|1|2|3|

Произведем построение такого массива значений префикс функции для строки, являющейся объединением паттерна и текста, разделенных символом, не входящим не в одну из этих строк. Тогда символы, для которых значение длины префикс функции будет совпадать с длиной паттерна будут являться последними символами вхождения паттерна в текст.

|$S$    |a|b|a|\$|a|b|a|b|a|
|-------|-|-|-|--|-|-|-|-|-|
|$|\Pi|$|0|0|1|0 |1|2|3|2|3|

При наивном вычислении длины префикс-функции необходимо потратить $O(|P|^2)$ опреаций.

In [2]:
def simple_prefix_length(text):
    result = 0
    for i in range(1, len(text) - 1):
        for j in range(i):
            if text[j] != text[-i + j]:
                break
        else:
            result = i
    return result

def simple_kmp(pattern, text):
    result = []
    search_string = pattern + '$' + text
    for i in range(1, len(search_string) + 1):
        if simple_prefix_length(search_string[:i]) == len(pattern):
            result.append(i - 2*len(pattern) - 1)
            
    return result

simple_kmp('aba', 'ababa')

[0, 2]

# Построение префикс-функции за линейное время

Будем последовательно искать значения префикс функции. На каждом шаге будем расматривать предыдущее значение длины префикс-функции. Сначала произведем проверку, можно ли увеличить предыдущий префикс-суффикс, добавив текущий символ. 

***Утверждение.*** Добавление символа в строку увеличивает длину ее префикс-функции, если добавляемый символ совпадает с символом, расположенным на индексе строки, равном текущей длине префикс-функции.

Если символ в тексте по индексу, равному предыдущему значению длины префикс-функции, не совпадает с текущим символом текста, то происходит нахождение меньшего префикс-суффикса. 

***Свойство.*** Все возможные суффикс-префиксы можно найти рекурсивным поиском максимальных суффикс-префиксов в текущем максимальныом суффикс-префиксе.  
$\Pi(\text{'ababa'}) = \text{'aba'}$  
$\Pi(\text{'aba'}) = \text{'a'}$  
$\Pi(\text{'a'}) = \text{''}$  

Исходя из описанного свойства можно понять, что длина меньшего префикс-суффикса уже вычислена и находится на индексе, равном текущему значению префикс-функции - 1. Данные действия продолжаются, пока не найдется префикс-суффикс, который можно расширить текущим символом, или пока длина префикс-функции не будет равно 0.


In [10]:
def kmp(pattern, text):
    result = []
    search_string = pattern + '$' + text
    prefix_length = [None] * len(search_string)
    current_prefix_length = 0
    prefix_length[0] = current_prefix_length
    for i in range(1, len(search_string)):
        while current_prefix_length and search_string[current_prefix_length] != search_string[i]:
            current_prefix_length = prefix_length[current_prefix_length - 1]
        
        if search_string[current_prefix_length] == search_string[i]:
            current_prefix_length += 1
        prefix_length[i] = current_prefix_length
        
        if current_prefix_length == len(pattern):
            result.append(i - 2*len(pattern))
    
    return result

kmp('aba', 'ababa')

[0, 2]

# Оценка сложности

***Утверждение.*** За один ход алгоритма величина длины префикс-функции возрастает не более, чем на 1.

Рассмотрим график длины префикс-функции от количества рассмотренных символов.

![](./images/kmp.png)

За один проход по строке длина префикс-функции не может превышать длины строки, следовательно суммарно за весь проход по строке длина префикс-функции за один ход также не может быть уменьшена больше, чем на длину строки. Следовательно данный алгоритм имеет линейную сложность $O(|P| + |T|)$.
